# TF-IDF

根据词频提取出《浅谈能源文化》文章的关键字

In [ ]:
import jieba
import pandas as pd
# 1、读取文章《浅谈能源文化》
with open('../data/浅谈能源文化.txt', 'r') as f:
    txt = f.read()
# 2、分词
txt = txt.split()
words = [jieba.lcut(x) for x in txt]
# 3、去除停用词
with open('../data/stoplist.txt', 'r', encoding='utf-8') as f:
    stop = f.read()
stop = stop.split()
words2 = [[i for i in x if i not in stop] for x in words]
# 4、统计词频
a = [' '.join(x) for x in words2]
b = ' '.join(a)
cipin = pd.Series(b.split()).value_counts()
# 5、提取词频最多的5个词语作为文章的关键字
cipin.index[:5]
key_words = cipin.iloc[:5].index
print(key_words)

改进：根据词频-逆文档概率提取《浅谈能源文化》的关键字

In [ ]:
with open('../data/字典.txt', 'r', encoding='utf-8') as f:
    dic = f.readlines()
print(dic[:4])
dic2 = [i.split()[:2] for i in dic]
print(dic2[:4])
dic3 = {i[0]:i[1] for i in dic2}

In [ ]:
n = 100000
import math
word3 = []
tf_idf3 = []
tf3 = []
idf3 = []
for (k, w) in zip(cipin[:10], cipin[:10].index):
    tf = k
    idf = math.log(n/(int(dic3.get(w, 0))+1))
    tf_idf = tf*idf
    word3.append(w)
    tf_idf3.append(tf_idf)
    tf3.append(tf)
    idf3.append(idf)
    print(f'词语 {w} 在文章中的词频为{k}, 在语料库中出现的次数为{dic3.get(w, 0)}，tf-idf是{tf_idf}')

In [ ]:
data = pd.DataFrame({'words':word3, 'tf':tf3, 'idf':idf3, 'tf-idf':tf_idf3})
data.sort_values(by='tf-idf', ascending=False)

# 文本聚类

对手机评论数据进行聚类

In [ ]:
import pandas as pd
f = open('../data/vivo数据.csv', encoding='utf-8')
data = pd.read_csv(f)
data.head()

In [ ]:
content = data['content']
content.head()

分词

In [ ]:
import jieba
content = content.apply(jieba.lcut)
content.head()

去除停用词

In [ ]:
with open('../data/stoplist.txt', 'r', encoding='utf-8') as f:
    stop = f.read()
stop = stop.split()
content = content.apply(lambda x: [i for i in x if i not in stop])
content.head()

转换为文本-词条矩阵

In [ ]:
x = content.apply(lambda x: ' '.join(x))
y = data['lab']
print(x.head(), y.head())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_data = cv.fit_transform(x)
cv.vocabulary_  # 字典
cv.get_feature_names()  # 词条
x_data.toarray()

使用KMeans聚类

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=5).fit(x_data)
model.labels_

In [ ]:
y_pre = model.labels_

评价聚类效果

In [ ]:
from sklearn.metrics import silhouette_score
# help(silhouette_score)
silhouette_score(x_data, y_pre)

In [ ]:
for i in range(3, 9):
    model = KMeans(n_clusters=i).fit(x_data)
    y_pre = model.labels_
    print(f'{i} {silhouette_score(x_data, y_pre)}')

# 情感分析

# 文本分类

In [1]:
import pandas as pd
f = open('G:/CDO/NLP/自然语言处理技术与应用/自然语言处理技术/06.数据/message80W1.csv', encoding='utf-8')
data = pd.read_csv(f)
data.head()

,1,0,商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一
0,2,1,南口阿玛施新春第一批限量春装到店啦   春暖花开淑女裙、冰蓝色公主衫 ...
1,3,0,带给我们大常州一场壮观的视觉盛宴
2,4,0,有原因不明的泌尿系统结石等
3,5,0,23年从盐城拉回来的麻麻的嫁妆
4,6,0,感到自减肥、跳减肥健美操、


In [9]:
content = data['商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一']
content

0         南口阿玛施新春第一批限量春装到店啦         春暖花开淑女裙、冰蓝色公主衫  ...
1                                          带给我们大常州一场壮观的视觉盛宴
2                                             有原因不明的泌尿系统结石等
3                                           23年从盐城拉回来的麻麻的嫁妆
4                                             感到自减肥、跳减肥健美操、
                                ...                        
799994                                    助排毒缓解痛经预防子宫肌瘤&amp
799995                                     这是今年首次启动I级防台应急响应
799996                                       丽江下飞机时迎接我们的是凉风
799997                              费了半天劲各种找关系终于联系上心仪公司的内部人
799998                                      是汉奸还是被强奸自己对号入座吧
Name: 商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一, Length: 799999, dtype: object

In [14]:
import jieba
content = content.apply(lambda x: list(jieba.cut(x)))
content

TypeError: expected string or bytes-like object

In [25]:
with open('../data/stoplist.txt', 'r', encoding='utf-8') as f:
    stop = f.read()
stop = stop.split()
stop = stop + ['\n', ' ']
content = content.apply(lambda x: [i for i in x if i not in stop])
content.head()

0    [指纹, 闹着玩, vivox7plus, 太, 老火, 18, 号, 买, 部, 只到, ...
1                 [久, 部坏机, 右手, 指纹, 解锁, 不行, 左手, 解锁, 成功]
2    [第一次, 京东, 买手机, 买, 完, 后悔, 物流, 慢, 关键, 包装, 简单, 收到...
3    [手机, 刚用, 天, 屏幕, 不好, 京东, 买, vivo, 实体店, 退换货, 京东,...
4    [25, 说, 到手, 26, 拿到, 手, 客服, 解决不了, 只会, 说, 抱歉, 手机...
Name: content, dtype: object

In [26]:
x = content.apply(lambda x: ' '.join(x))
y = data['lab']
print(x.head(11), y.head())

0     指纹 闹着玩 vivox7plus 太 老火 18 号 买 部 只到 一部 hellip h...
1                            久 部坏机 右手 指纹 解锁 不行 左手 解锁 成功
2     第一次 京东 买手机 买 完 后悔 物流 慢 关键 包装 简单 收到 手机 装 手机 盒子 ...
3     手机 刚用 天 屏幕 不好 京东 买 vivo 实体店 退换货 京东 拿走 做 鉴定 太 麻...
4     25 说 到手 26 拿到 手 客服 解决不了 只会 说 抱歉 手机 想象 中要 厚 指纹 ...
5                                  触屏 不灵 反应迟钝 app 卡死 再评
6     拿到 手 惊艳 iPhone 用户 屏幕 颜色 显示 别扭 开 护眼 模式 舒服 地方 说 ...
7     一星 致敬 vivo 自营 碎 屏险 三个 供应商 客服 三个 说法 态度 高冷 发货 时间...
8     先说 说 一颗 星 京东 发货 第一批 10 分钟 全款 预定 拖 星期 发货 说好 送 半...
9     手机 边框 边框 拐角 手机 屏幕 边框 合拢 缝隙 闪光灯 反光 描述 心情 好多天 后悔...
10    完 完 中奖 缝 价位 品控 无语 手机 真的 好看 真 瓶 没得说 满意 屏幕 清晰 xp...
Name: content, dtype: object 0    1
1    1
2    1
3    1
4    1
Name: lab, dtype: int64


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_data = cv.fit_transform(x)
cv.vocabulary_  # 字典
cv.get_feature_names()  # 词条
x_data.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data.toarray(), y, test_size=0.2, random_state=123)
model = GaussianNB().fit(x_train, y_train)
y_pre = model.predict(x_test)

In [29]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pre))
print(classification_report(y_test, y_pre))

[[  9   0   3   1   8]
 [  1   0   0   0   2]
 [  0   0   3   0   7]
 [  0   0   0   9   3]
 [ 17  10  25  16 367]]
             precision    recall  f1-score   support

          1       0.33      0.43      0.38        21
          2       0.00      0.00      0.00         3
          3       0.10      0.30      0.15        10
          4       0.35      0.75      0.47        12
          5       0.95      0.84      0.89       435

avg / total       0.88      0.81      0.84       481



# 使用FastText分类

In [69]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=123)

In [70]:
with open('../tmp/train.txt', 'w', encoding='utf-8') as f:
    for i in x_train.index:
        f.write(x_train[i]+ ' __label__' + str(y_train[i]) +'\n')

In [71]:
import fastText
classifier = fastText.train_supervised(r'..\tmp\train.txt')

In [72]:
classifier.get_words()[:12]

['</s>', '手机', '屏幕', '不错', '摄像头', '指纹', '感觉', '屏', 'vivo', '买', '说', '喜欢']

In [73]:
classifier.get_labels()[:12]

['__label__5', '__label__1', '__label__3', '__label__4', '__label__2']

In [74]:
y_pre = classifier.predict(list(x_test))
y_pre = [int(x[-1]) for x in y_pre[0]]

In [75]:
from sklearn.metrics import confusion_matrix, classification_report
y_test = list(y_test)
print(confusion_matrix(y_test, y_pre))
print(classification_report(y_test, y_pre))

[[  0   4   0   0  17]
 [  0   0   0   0   3]
 [  0   0   0   0  10]
 [  0   0   0   0  12]
 [  0   0   0   0 435]]
             precision    recall  f1-score   support

          1       0.00      0.00      0.00        21
          2       0.00      0.00      0.00         3
          3       0.00      0.00      0.00        10
          4       0.00      0.00      0.00        12
          5       0.91      1.00      0.95       435

avg / total       0.82      0.90      0.86       481



C:\Users\45543\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
